# Cylinder Test Case

## Boilerplate

In [84]:
import numpy as np
import scipy as sc
import pandas as pd

from numba import njit

from pathlib import Path
from dataclasses import dataclass

# Plotting
import bokeh
import bokeh.palettes as palettes

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.layouts import Column, gridplot

palette = palettes.Category10[10]

output_notebook()

Loading BokehJS ...

In [2]:
%%html
<!-- So that markdown cells don't stretch across the whole screen -->
<style>
.jp-RenderedMarkdown {
    max-width: 700px;
}
.jp-MarkdownCell {
    max-width: 900px;
}
</style>

In [3]:
# Generic plot settings

line_settings = dict(
    line_width=2,
    muted_alpha=0.1
)

fig_settings = dict(
    width=500,
    height=350
)

tool_settings = dict(
    toolbar_location=None,
    tools=[]
)

# Issues

- [x] Need resonant frequencies for all modes to get the fractional frequency shift
    - Resonant frequencies aren't actually required since the perturbation theory naturally provides the fractional shift without extra effort.
- [x] Need device mass
    - Not required since COMSOL sets the modal masses to zero.
- [x] Need to understand the relationship between conventional analysis and modal mass normalisation
    - Done
- [ ] Vectorise MeshInterpolation class call

## Interpretting COMSOL output

In [199]:
def count_header_lines(path):
    """Return the number of header lines for COMSOL .csv file"""
    return len(extract_header_lines(path))


def extract_header_lines(path):
    """Return the header lines for a COMSOL .csv file without prefix"""
    header_lines = []
    
    with open(path) as f:
        for line in f.readlines():
            if line.startswith('%'):
                header_lines.append(line[2:])
                    
            else:
                break
        return header_lines


def extract_field_names(path):
    """Return field names for a COMSOL .csv file"""
    last_header_line = extract_header_lines(path)[-1]
    field_names = [
        entry.strip()
        for entry in last_header_line.split(',')
    ]
    return field_names


def extract_dataframe(path):
    n_headers = count_header_lines(path)
    field_names = extract_field_names(path)
    
    df = pd.read_csv(path, skiprows=n_headers, header=None, names=field_names)
    return df
    
    
def extract_eigenmodes(path, mode_dim=3):
    """Parse COMSOL .csv and return eigenmodes structure"""
    df = extract_dataframe(path)
    
    pt_dim = 3
    n_fields = len(df.columns)
    n_pts = len(df)
    
    n_modes = (n_fields - pt_dim) // mode_dim
    if n_modes * mode_dim + pt_dim != n_fields:
        raise ValueError('Field number mismatch!')
        
    pts = np.array(df.iloc[:, 0:pt_dim])
    modes = np.empty((n_pts, n_modes, mode_dim))
    
    for i in range(n_modes):
        m_idx = pt_dim + mode_dim * i
        modes[:, i, :] = np.array(df.iloc[:, m_idx:m_idx + mode_dim])
    
    result = Eigenmodes(pts, modes)
    return result

    
@dataclass
class Eigenmodes:
    pts:np.ndarray
    modes:np.ndarray
    
    
@njit
def triangle_pcoordinates(p, ps):
    """Return triangle coordinates (xi1, xi2, xi3) for point p in triangle (p1, p2, p3)
    
    Parameters
    ----------
    p : (d,) ndarray
        point in triangle
    ps : (3, d) ndarray
        points as rows of the matrix
    
    Returns
    -------
    (3,) ndarray
        triangular coordinates
    """
    A = np.empty((3, 2))
    coords = np.empty(3)
    
    A[:, 0] = p1 - p3
    A[:, 1] = p2 - p3
    b = p - p3
    
    coords[:2] = np.linalg.lstsq(A, b, rcond=False)[0]
    coords[2] = 1 - coords[0] - coords[1]
    return coords


#@njit
def triangle_interpolate(p, ps, us):
    """Return linear interpolation at point p given values at nodes
    
    Point is not projected onto the mesh
    
    Parameters
    ----------
    p : (d,) ndarray
        point in triangle
    ps : (3, d) ndarray
        points as rows of the matrix
    us : (3, df) ndarray
        values of function at triangle nodes
        
    Returns
    -------
    (df,) ndarray
        value of df-dimensional linear interpolation at point p
    """
    coords = np.linalg.lstsq(ps.T, p, rcond=False)[0]
    
    dims = np.ones(us.ndim, int)
    dims[0] = -1
    
    value = np.sum(coords.reshape(dims) * us, axis=0)
    return value
        
    
class MeshInterp:
    """Structure for linear interpolation over mesh points
    
    The values at each mesh point can be an arbitrary array
    
    Parameters
    ----------
    points : (n, d) ndarray
        points as rows of the matrix
    mesh_values : (n, *dfs) ndarray
        values of function at mesh nodes
    """
    
    def __init__(self, points, mesh_values):
        # Check Arguments
        
        if mesh_values.ndim < 2:
            raise ValueError('mesh_values should be at least 2-dimensional')
        
        n_mesh_values, *mesh_df = mesh_values.shape
        n_pts, pt_dim = points.shape
            
        if n_pts != n_mesh_values:
            raise ValueError("size of first dimension of mesh_values should match num. pts.")
        
        # Store data & build KDTree
        self._points = points
        self.tree = sc.spatial.KDTree(points)
        self.mesh_values = mesh_values
        self.mesh_df = mesh_df
        self.pt_dim = pt_dim
    
    def __call__(self, ps):
        """Return function on mesh at points ps
        
        Linear interpolation is used to compute the value of the function value 
        at the point.
        
        Parameters
        ----------
        ps : (m, pt_dim) ndarray 
            points to project and evaluate modes at
        
        Returns
        -------
        values : (m, *mesh_df) ndarray
            values of function on the mesh
        """
        
        # Check argument
        if ps.ndim != 2:
            raise ValueError('point array should have 2 dimensions')
        elif ps.shape[1] != self.pt_dim:
            raise ValueError(f'points should have dimension {self.pt_dim}')
        
        m, _ = ps.shape
        values = np.empty((m, *self.mesh_df))
        
        for i in range(m):
            p = ps[i, :]
            _, idxs = self.tree.query(p, k=3)
            
            mesh_pts = self._points[idxs, :]
            mesh_pts_us = self.mesh_values[idxs, ...]
            values[i, :] = triangle_interpolate(p, mesh_pts, mesh_pts_us)
        
        return values

In [200]:
# Testing
p = np.array([0.0, 1.0, 0.0])

p1 = np.array([0.0, 0.0, 0.0])
p2 = np.array([1.0, 0.0, 0.0])
p3 = np.array([0.0, 1.0, 0.0])

ps = np.vstack((p1, p2, p3))

us = np.array([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
])

us = np.random.randn(3, 5, 5)

triangle_coordinates(p, ps)
triangle_interpolate(p, ps, us)

array([[-0.15101356, -0.93508339, -0.1957028 , -2.063191  ,  0.62551217],
       [ 0.33381267,  0.3483876 ,  1.84758393,  1.32531759,  0.93246781],
       [ 0.10540109,  0.28653676, -1.28865761,  0.62323011,  0.81749789],
       [-0.43835262,  0.78914401, -0.39850431,  0.75819972, -0.43105662],
       [-0.44241667, -2.06648394,  1.0762709 ,  0.32991514,  0.39215753]])

In [204]:
path = Path('mm_surface_disp.csv')
df = extract_dataframe(path)
analysis = extract_eigenmodes(path)

pts = analysis.pts
us = analysis.modes

mint = MeshInterp(pts, us)

zs = np.linspace(0, 500, 600)
pts = np.array([[0.0, 10, z] for z in zs])
values = mint(pts)

In [216]:
p = figure()
p.line(zs, values[:, 3, 1] / 1.2e18, line_width=2)
show(p)

In [ ]:
u1 = np.array([1.0, 1.0])
u2 = np.array([1.0, -1.0])



In [2]:
!head -20 mm_surface_disp.csv

% Model,cyl_test.mph
% Version,COMSOL 6.1.0.252
% Date,"May 11 2023, 17:25"
% Dimension,3
% Nodes,9072
% Expressions,18
% Description,"Displacement field, X-component, Displacement field, Y-component, Displacement field, Z-component"
% Length unit,nm
% X,Y,Z,u (nm) @ lambda=1,v (nm) @ lambda=1,w (nm) @ lambda=1,u (nm) @ lambda=2,v (nm) @ lambda=2,w (nm) @ lambda=2,u (nm) @ lambda=3,v (nm) @ lambda=3,w (nm) @ lambda=3,u (nm) @ lambda=4,v (nm) @ lambda=4,w (nm) @ lambda=4,u (nm) @ lambda=5,v (nm) @ lambda=5,w (nm) @ lambda=5,u (nm) @ lambda=6,v (nm) @ lambda=6,w (nm) @ lambda=6
-8.311766015409734,-5.560072792326597,500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
-7.729323821546742,-6.344884022712318,498.2985901881011,2.6125742099171955E15,1.7998863883885618E15,-4.870834455248633E15,4.758509056166872E14,1.3158871464862868E15,-1.8467057212053208E15,7.202037207540747E15,5.308787173364463E15,-1.3342291877105766E16,1.8234965484401012E15,3.1361157812196215E15,-5.060544357983477E15,1.4008857575509446E1

In [3]:
!head -20 surface_disp.csv

% Model,cyl_test.mph
% Version,COMSOL 6.1.0.252
% Date,"May 11 2023, 15:33"
% Dimension,3
% Nodes,9072
% Expressions,18
% Description,"Displacement field, X-component, Displacement field, Y-component, Displacement field, Z-component"
% Length unit,nm
% X,Y,Z,u (nm) @ lambda=1,v (nm) @ lambda=1,w (nm) @ lambda=1,u (nm) @ lambda=2,v (nm) @ lambda=2,w (nm) @ lambda=2,u (nm) @ lambda=3,v (nm) @ lambda=3,w (nm) @ lambda=3,u (nm) @ lambda=4,v (nm) @ lambda=4,w (nm) @ lambda=4,u (nm) @ lambda=5,v (nm) @ lambda=5,w (nm) @ lambda=5,u (nm) @ lambda=6,v (nm) @ lambda=6,w (nm) @ lambda=6
-8.311766015409734,-5.560072792326597,500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
-7.729323821546742,-6.344884022712318,498.2985901881011,2741163.720100259,1888475.768358216,-5110574.264324711,499272.19520053273,1380654.4798352253,-1937599.853084515,7546645.989795721,5562806.228609937,-1.3980704430074016E7,1910773.120422019,3286217.21091325,-5302753.189299614,1.468193552396545E7,1.149375105395601E7,-2.703467589610034E